In [ ]:
#| default_exp chatbot

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox, Stack, Layout
from traitlets import HasTraits
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
from jupyter_mentor.llm import FileModel

In [ ]:
class ChatBotModel(FileModel):

    bot_identity = traitlets.Unicode()
    human_identity = traitlets.Unicode()

    def __init__(self, bot_template="", human_template="{input_text}"):
        super().__init__()
        self.bot_identity = "You are playing the role of a tutor/educator"
        self.human_identity = "I am a student"
        self.update_bot_template(bot_template)
        self.update_human_template(human_template)

    def update_bot_template(self, bot_template):
        self.bot_message_prompt = SystemMessagePromptTemplate.from_template("\n\n".join([self.bot_identity, bot_template]))

    def update_human_template(self, human_template):
        self.human_message_prompt = HumanMessagePromptTemplate.from_template("\n\n".join([self.human_identity, human_template]))

    def prompt(self, input_text):
        ret = self.llm.invoke(input_text)
        return ret.content
    
    def prompt_with_template(self, kwargs):
        self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])   
        ret = self.llm.invoke(self.chat_prompt.format_prompt(**kwargs))
        return ret.content

    def prompt_course_files(self, input_text):
        if self.db:
            try:
                docs = self.db.similarity_search(input_text, k=1)
                docs_content = " ".join([d.page_content for d in docs])
                self.update_bot_template(self.bot_identity + "\n\nAnswer the question based on the following information: {docs}\n\n{input_text}")
                self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])   
                ret = self.llm.invoke(self.chat_prompt.format_prompt(docs=docs, input_text=input_text)) 
                return "\n\n".join([ret.content + '\n\nSourced from ' + docs[0].metadata['source'] + ': \n', docs_content])
            except Exception as e: 
                return e
        else:
            return self.prompt(input_text)

In [ ]:
chatbot_model = ChatBotModel()
#chatbot_model.prompt('hello')
#chatbot_model.prompt_with_template({'input_text': 'hi'})

In [ ]:
chatbot_model.load_pdf_to_db("course_files/STP 420 spring 2024 course syllabus.pdf")

In [ ]:
chatbot_model.prompt_course_files("when will we learn about integration?", )

In [ ]:
class ChatBotView(widgets.VBox):
    
    def __init__(self):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat = Textarea(
            disabled = True,
            layout=Layout(width='90%', height='400px')
        )
        self.user_input_and_submit = HBox()
        self.user_input = widgets.Text(
            placeholder='Message AI chatbot...',
            #layout=Layout(width='100%')
        )
        self.submit_button = widgets.Button(
            value=False,
            disabled=False,
            button_style='success',
            icon='arrow-circle-right' 
        )
        self.user_input_and_submit.children = (self.user_input, self.submit_button)

        self.children = (self.chat, self.user_input_and_submit) 

In [ ]:
chatbot_view = ChatBotView()
chatbot_view

In [ ]:
class ChatBot(ChatBotView):

    def __init__(self, model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()
        self.submit_button.on_click(self.on_click)
        self.model =model

    def on_click(self, change):
        self.chat.value = self.chat.value + "USER: " + self.user_input.value + '\n\n'
        self.user_input.value = ''
        ret = self.model.prompt_course_files(self.user_input.value)
        self.chat.value = self.chat.value +  "CHATBOT: "  + ret + '\n\n'

In [ ]:
chatbot = ChatBot(chatbot_model)
chatbot

In [ ]:
file_model = FileModel()
file_model.load_pdf_to_db("course_files/STP 420 spring 2024 course syllabus.pdf") #file as input

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()